Research Analysis performed by Kevin Kwok-Ho Liu, University of California Berkeley

A joint collaboration between the Division of Data Science, Computing, & Society and Cal Alumni Association


In [1]:
import numpy as np
import pandas as pd 
from random import choices

import bz2 # Used to read compressed data
import os # Used to interact with the file system

We begin by loading the appropriate data tables.

In [2]:
interest_info = pd.read_csv("./data_file/Interest_Info.csv")
caa_membership = pd.read_csv("./data_file/CAA_Membership_Info.csv")
Student_Activity = pd.read_csv("./data_file/Student_Activity_Info.csv")
affilation_info = pd.read_csv("./data_file/Affiliation_Info.csv")

In [3]:
affilation_info.head()

entity_id affil_code                    affil_code_desc affil_level_code  \
0          2       LHSM    Lawrence Hall of Science Member                M   
1          2         V6          Guardsmen - San Francisco                M   
2          2         V2  Junior League of Oakland/East Bay               AM   
3          2        S38             Claremont Country Club                M   
4          2      ATFAT               Friends of Athletics               FR   

  affil_level_code_desc affil_status_code affil_status_desc  \
0                Member                 F            Former   
1                Member                 C           Current   
2      Associate Member                 C           Current   
3                Member                 C           Current   
4                Friend                 C           Current   

            start_date            stop_date  
0  2013-08-20 00:00:00  2014-08-31 00:00:00  
1  1995-07-01 00:00:00                  NaN  
2  1994-06-15 00:00:00                  NaN  
3  1994-04-06 00:00:00                  NaN  
4  2017-09-18 00:00:00                  NaN

Let's start by inporting some data, we will focus on the interest_info, CAA_Membership_Info, and Student_Activity_Info files to see what we can find out about a student's interests and their participation of
the CAA network.

Here we have the Student Activity table that has the following column values:
entity_id,student_activity_code, student_activity_desc,student_particip_desc 	activity_office_cod)

In [4]:
Student_Activity.head() 
#Student_Activity = Student_Activity.drop(columns=['activity_office_desc'])
Student_Activity.head()

entity_id student_activity_code            student_activity_desc  \
0          3                  GAOI                 Alpha Omicron Pi   
1          4                  GSCF             Sigma Chi Fraternity   
2          4                  USPS  Assoc Students Of Pretrial Svcs   
3          4                  RHI6         1963-72 I-House Resident   
4          4                  RHIH     International House Resident   

  student_particip_code student_particip_desc activity_office_code  \
0                     P          Participated                        
1                     P          Participated                        
2                     P          Participated                        
3                     P          Participated                        
4                     P          Participated                        

  activity_office_desc start_dt stop_dt  
0                  NaN      NaN     NaN  
1                  NaN      NaN     NaN  
2                  NaN      NaN     NaN  
3                  NaN      NaN     NaN  
4                  NaN      NaN     NaN

We also have the caa_membership table with the following column values:entity_id 	caa_current_memb 	caa_latest_memb_type_code 	caa_latest_memb_type_desc 	caa_latest_memb_status_code 	caa_latest_memb_status_desc 	caa_latest_memb_year 	caa_latest_memb_start_date 	caa_latest_memb_stop_date

In [5]:
caa_membership.head() 

entity_id caa_current_memb caa_latest_memb_type_code  \
0          3                Y                        LB   
1          4                Y                        RL   
2          7                Y                        RL   
3         10                Y                        RL   
4         11                Y                        RL   

      caa_latest_memb_type_desc caa_latest_memb_status_code  \
0  CAA Golden Bear Life Upgrade                           P   
1       CAA Regular Life Member                           P   
2       CAA Regular Life Member                           P   
3       CAA Regular Life Member                           P   
4       CAA Regular Life Member                           P   

  caa_latest_memb_status_desc  caa_latest_memb_year  \
0                        Paid                  2012   
1                        Paid                  1973   
2                        Paid                  1931   
3                        Paid                  1975   
4                        Paid                  1972   

  caa_latest_memb_start_date caa_latest_memb_stop_date  
0        2012-01-06 00:00:00                       NaN  
1        1972-03-13 00:00:00                       NaN  
2        1930-01-01 00:00:00                       NaN  
3        1974-01-17 00:00:00                       NaN  
4        1971-05-16 00:00:00                       NaN

And we have the interest_info table with the follow column values:
    entity_id
    interest_code
    interest_desc
    start_dt 
    stop_dt

In [6]:
interest_info.head() 

entity_id interest_code        interest_desc             start_dt  \
0          2           GOL                 Golf                  NaN   
1          2           FOO             Football                  NaN   
2         15           DEM  Democratic Politics  2019-01-01 00:00:00   
3         15           CIV        Civic Affairs  2019-01-01 00:00:00   
4         18           GOL                 Golf  2015-03-24 00:00:00   

               stop_dt  
0                  NaN  
1  2015-02-05 00:00:00  
2                  NaN  
3                  NaN  
4                  NaN

We perform an outer merge with caa_membership and interest_info to get a table of all members and what interests they
had. We also clean the data up a bit by removing any members that have a NaN of "caa_current_memb" status.

In [7]:
## Interest info outer merge with caa_membership
membership_interests = interest_info.merge(caa_membership, how = "outer", left_on = "entity_id", right_on = "entity_id")
membership_interests = membership_interests[membership_interests['caa_current_memb'].notnull()]
membership_interests.head()

entity_id interest_code        interest_desc             start_dt stop_dt  \
2          15           DEM  Democratic Politics  2019-01-01 00:00:00     NaN   
3          15           CIV        Civic Affairs  2019-01-01 00:00:00     NaN   
8          29           REP  Republican Politics  2019-01-01 00:00:00     NaN   
9          29           CIV        Civic Affairs  2019-01-01 00:00:00     NaN   
10         31           DEM  Democratic Politics  2019-01-01 00:00:00     NaN   

   caa_current_memb caa_latest_memb_type_code caa_latest_memb_type_desc  \
2                 N                        PL           CAA Paying Life   
3                 N                        PL           CAA Paying Life   
8                 Y                        RL   CAA Regular Life Member   
9                 Y                        RL   CAA Regular Life Member   
10                Y                        RL   CAA Regular Life Member   

   caa_latest_memb_status_code caa_latest_memb_status_desc  \
2                            W                      Lapsed   
3                            W                      Lapsed   
8                            P                        Paid   
9                            P                        Paid   
10                           P                        Paid   

    caa_latest_memb_year caa_latest_memb_start_date caa_latest_memb_stop_date  
2                 2010.0        2010-02-10 00:00:00                       NaN  
3                 2010.0        2010-02-10 00:00:00                       NaN  
8                 1961.0        1960-06-02 00:00:00                       NaN  
9                 1961.0        1960-06-02 00:00:00                       NaN  
10                1996.0        1995-10-13 00:00:00                       NaN

In [8]:
membership_interests_filtered = membership_interests[['entity_id','interest_code','interest_desc','caa_current_memb','caa_latest_memb_type_desc']]
membership_interests_filtered.head()

entity_id interest_code        interest_desc caa_current_memb  \
2          15           DEM  Democratic Politics                N   
3          15           CIV        Civic Affairs                N   
8          29           REP  Republican Politics                Y   
9          29           CIV        Civic Affairs                Y   
10         31           DEM  Democratic Politics                Y   

   caa_latest_memb_type_desc  
2            CAA Paying Life  
3            CAA Paying Life  
8    CAA Regular Life Member  
9    CAA Regular Life Member  
10   CAA Regular Life Member

We now filter out only the current members.

In [9]:
membership_interests_filtered_member_only = membership_interests_filtered[membership_interests_filtered['caa_current_memb'] == "Y"] 
membership_interests_filtered_member_only.head()

entity_id interest_code              interest_desc caa_current_memb  \
8          29           REP        Republican Politics                Y   
9          29           CIV              Civic Affairs                Y   
10         31           DEM        Democratic Politics                Y   
11         31           CIV              Civic Affairs                Y   
12         31           FLY  Flying/Airplane Ownership                Y   

   caa_latest_memb_type_desc  
8    CAA Regular Life Member  
9    CAA Regular Life Member  
10   CAA Regular Life Member  
11   CAA Regular Life Member  
12   CAA Regular Life Member

We now see which interests had the most members associated with it.

In [10]:
interest_sorted = membership_interests_filtered.groupby('interest_desc').count()[['entity_id']].rename(columns = {"entity_id":"count"}).sort_values('count',ascending = False)
interest_sorted = interest_sorted.reset_index()
interest_sorted.head()

interest_desc  count
0        Civic Affairs  19139
1  Democratic Politics  16149
2  Republican Politics   3906
3       Aquatic Sports   1555
4               Health   1448

We do the same analysis on student organizations. 

In [11]:
student_activity_and_membership = Student_Activity.merge(caa_membership, how = "outer", left_on = "entity_id", right_on = "entity_id")
student_activity_and_membership_sorted = student_activity_and_membership.groupby("student_activity_desc").count().rename(columns = {"entity_id":"count"}).sort_values('count',ascending = False)
student_activity_and_membership_sorted[['count']].head()

count
student_activity_desc                          
SLCP-Student Participants                 14744
International House Resident              12207
Cal Corps Friends and Alumni              11317
Undergrad Research Apprentice Pgm (URAP)  10509
Honor Students' Society                    4730

We now perform a hypothesis test to see if members are more likely to have certain interests. To do this, we will use a bootstrap methdology. We will look at each interest and see if it is associated with membership participation. 

Our null hypothesis is:
In the population, the percentage of alumni being active members interested in democratic policies is the same as alumns who don't have particular democratic policies. The difference in the sample is due to random chance.

Likewise our alternative hypothesis is:
In the population, alums who have an interest in democratic policies are more likely to participate on average, than alums who don't have these interests. 

And our test statistic would be:
The alternative hypothesis compares the percentage of alumni participating as members of the two groups (interest/no interest) and says that the average for the alumni who have this interest is greater. Therefore it is reasonable for us to use the difference between the two group means as our statistic.
 
 

There are 352 different interests. We store each of the in an array_of_interests

In [12]:
len(interest_sorted["interest_desc"].unique())
array_of_interests = interest_sorted["interest_desc"].array
array_of_interests

<PandasArray>
[                     'Civic Affairs',                'Democratic Politics',
                'Republican Politics',                     'Aquatic Sports',
                             'Health',             'Diverse Constituencies',
                           'Politics',                   'Entrepreneurship',
                          'Education',                              'Women',
 ...
 'Distance Learning/Online Education',                        'Early Music',
                             'Ethics',                         'Freshwater',
     'Spanish & Portuguese Interests',              'Palestinian Interests',
          'Internet-Related e-Health',                               'K-12',
                    'Lutheran Church',                      'Public Health']
Length: 351, dtype: object

As a proof of concept, let's just focus on democratic policies first. 

In [13]:
democratic_policies = membership_interests[membership_interests["interest_desc"] == "Democratic Politics"]
democratic_participate = democratic_policies[democratic_policies["caa_current_memb"] == "Y"]
democratic_no_participate = democratic_policies[democratic_policies["caa_current_memb"] == "N"]
democratic_policies.head()

entity_id interest_code        interest_desc             start_dt stop_dt  \
2          15           DEM  Democratic Politics  2019-01-01 00:00:00     NaN   
10         31           DEM  Democratic Politics  2019-01-01 00:00:00     NaN   
20         80           DEM  Democratic Politics  2019-01-01 00:00:00     NaN   
24         82           DEM  Democratic Politics  2015-03-10 00:00:00     NaN   
40        147           DEM  Democratic Politics  2019-01-01 00:00:00     NaN   

   caa_current_memb caa_latest_memb_type_code caa_latest_memb_type_desc  \
2                 N                        PL           CAA Paying Life   
10                Y                        RL   CAA Regular Life Member   
20                Y                        RL   CAA Regular Life Member   
24                Y                        RL   CAA Regular Life Member   
40                Y                        RL   CAA Regular Life Member   

   caa_latest_memb_status_code caa_latest_memb_status_desc  \
2                            W                      Lapsed   
10                           P                        Paid   
20                           P                        Paid   
24                           P                        Paid   
40                           P                        Paid   

    caa_latest_memb_year caa_latest_memb_start_date caa_latest_memb_stop_date  
2                 2010.0        2010-02-10 00:00:00                       NaN  
10                1996.0        1995-10-13 00:00:00                       NaN  
20                1931.0        1930-01-01 00:00:00                       NaN  
24                1972.0        1971-10-10 00:00:00                       NaN  
40                1951.0        1950-07-31 00:00:00                       NaN

As we can see, of the members who expressed an interest in democratic politics, there appears to be about ~1000 more members participating in CAA as members. However, this alone does not prove or show that CAA members are more left-leaning in general. In further analysis, we will attempt to awnser this question. 

In [14]:
print(len(democratic_participate) /len(democratic_policies))

0.5303733977336058


53% of alumns with democratic interests were CAA members, compared with 50% from the general population.

In [15]:
len(membership_interests_filtered_member_only)/len(membership_interests)

0.5037793034056677

Our test statistic is calculated as below. Greater values will suggest an anomoly.
Basically we are finding the difference in participation between members who have expressed an interest
in democratic policies versus the general population.

In [16]:
point_test_statistic = len(democratic_participate) /len(democratic_policies) - len(membership_interests_filtered_member_only)/len(membership_interests)
point_test_statistic

0.026594094327938134

That said, we will now perform A/B testing to see if people who are interested in democratic politics become CAA members due to random chance or if there is a correlation between the CAA membership and favorability of democratic politics. We shuffle the original memberships interest table (shuffle column caa_current_memb) and calculate the test statistic. 


In [20]:
array_of_test_statistics = []
sampling_array = membership_interests["caa_current_memb"].array

for i in range(0,1000):
    ##Sampling methodology: 
    #new_suffled_list = choices(sampling_array, k=len(membership_interests_filtered))
    new_suffled_list = np.array(membership_interests.sample(n=len(membership_interests),replace = True)["caa_current_memb"])
    membership_interests_filtered['caa_current_memb_shuffled'] = new_suffled_list
    shuffled_table = membership_interests_filtered
    
    democratic_policies_shuffled = shuffled_table[shuffled_table["interest_desc"] == "Democratic Politics"]
    democratic_participate_shuffled = democratic_policies_shuffled[democratic_policies_shuffled["caa_current_memb_shuffled"] == "Y"]
    democratic_no_participate_shuffled = democratic_policies_shuffled[democratic_policies_shuffled["caa_current_memb_shuffled"] == "N"]
    
    test_statistic =  (len(democratic_participate_shuffled) /len(democratic_policies_shuffled)) - 0.5037793034056677
    array_of_test_statistics.append(test_statistic)



/Users/kevinliu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
print(len(democratic_participate_shuffled), len(democratic_no_participate_shuffled), len(democratic_policies_shuffled))

8073 8076 16149


We have a p-value of 0, indicating that the null hypothesis could not have happened. Hence, we can conclude that alumns who expressed an interest in democratic politics are more likely to become CAA current members.

In [32]:
count = 0
for i in range(len(array_of_test_statistics)):
    if (point_test_statistic >= array_of_test_statistics[i]):
        count = count + 1

1 - count/1000

0.0

In [33]:
 array_of_test_statistics

[0.0020724521209902758,
 0.00040052197051665406,
 0.0031251488823996754,
 0.0023201454766160756,
 -0.003872188414027311,
 0.0020105287820838535,
 -0.0027575683137115448,
 -0.002571798296992167,
 0.001267448715206676,
 -0.004553345141998122,
 -0.0005902514519863233,
 0.00244399215442892,
 0.004425538999434764,
 0.005725929116469963,
 -0.0014571781966764563,
 0.0022582221377096534,
 0.0008959086817680317,
 0.0020724521209902758,
 -0.0064729686480975435,
 -0.0026956449748051226,
 0.0020724521209902758,
 -0.0035006483805887223,
 0.0012055253763002538,
 0.006221315827721341,
 0.004177845643809075,
 -0.006411045309191121,
 0.01154672297367465,
 -0.0003425580963606345,
 -0.005172578531062455,
 -0.007897205442945532,
 0.0020724521209902758,
 -0.0018906415690214118,
 -0.007401818731694043,
 -0.0002806347574542123,
 -0.004553345141998122,
 -0.004181805108559478,
 0.0003385986316101208,
 0.007212089250224318,
 0.0013912953930195204,
 0.0010197553595809872,
 0.005725929116469963,
 -0.0006521747908